In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from modules.Untangler import Untangler

random_seed = 50

In [ ]:
sample_count = 5
df = pd.read_csv("./data/GoldSet.csv")
df["Diff"] = df["Diff"].apply(lambda x: x.replace("\\ No newline at end of file","").strip())

buggy_df = df[df.Decision == 'Buggy'][["CommitMessage", "Diff", "Decision"]]
rows = np.random.choice(buggy_df.index.values,sample_count)
buggy_df = buggy_df.loc[rows]

notbuggy_df = df[df.Decision == 'NotBuggy'][["CommitMessage", "Diff", "Decision"]]
rows = np.random.choice(notbuggy_df.index.values,sample_count)
notbuggy_df = notbuggy_df.loc[rows]

few_shot_data = [*buggy_df.values.tolist() , *notbuggy_df.values.tolist()]
json.dump(few_shot_data,open("./data/FewShots.json","w"))

In [4]:
df = pd.read_csv("./data/GoldSet.csv")
#df["Diff"] = df["Diff"].apply(lambda x: x.replace("\\ No newline at end of file","").strip())

In [11]:
untangler = Untangler("openai", True)
untangler.prepare_prompt(df["CommitMessage"].iloc[250], df["Diff"].iloc[250])
untangler.print_prompt()

openai
Prompt for: openai

[{'role': 'developer', 'content': 'You are a Git commit review assistant. Given a Java source code diff and its commit message, analyze both to determine if the changes align with the described bug fix. Assess the relevance between the commit message and code modifications, identifying patterns such as error-handling updates, logical corrections, exception-handling improvements, and other indicators of bug-related changes. Use the provided examples as reference points to enhance accuracy in detecting bug-related changes.'}, {'role': 'user', 'content': 'Commit Messaage: Fix two bugs in MinMaxPriorityQueue (introduced in []). First is a bug in removeAt(int) that sometimes causes the wrong element to be removed. Second is a bug that sometimes causes certain elements to be iterated over more than once if elements were removed during iteration.\n\nReported externally at https://github.com/google/guava/issues/2658\n\n-------------\nCreated by MOE: https://github.co

In [12]:
df["Decision"].iloc[250]

'NotBuggy'

In [13]:
untangler.detect()

'The commit message "Fix Quality flaws" suggests that the changes made are intended to improve the quality of the code rather than specifically addressing a bug. The code diff shows a modification in how the `JsonWriter` is handled, changing it from a manual instantiation and closure to a try-with-resources statement. \n\nThis change improves resource management by ensuring that the `JsonWriter` is properly closed after use, which can prevent potential resource leaks. While this is a quality improvement and may indirectly prevent issues related to resource handling, it does not directly address any specific bugs or errors in functionality as described in the commit message.\n\nOverall, the changes align with improving code quality but do not indicate a direct bug fix. Therefore, the assessment is NotBuggy.'

In [ ]:
untangler = Untangler("gemini")
y_true, y_pred = untangler.batch_detect(df)
accuracy_score(y_true, y_pred)
print(classification_report(y_true, y_pred))

In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

       Buggy       0.72      0.95      0.82       144
    NotBuggy       0.91      0.56      0.69       122

    accuracy                           0.77       266
   macro avg       0.81      0.75      0.75       266
weighted avg       0.80      0.77      0.76       266



In [ ]:
untangler = Untangler("openai")
untangler.get_batch_result("batch_67ba5b015398819094fe8cc1b5928ae6")

In [155]:
untangler = Untangler("openai")

y_true = []
for index, row in df.iterrows():
    y_true.append(row["Decision"])
batch, y_pred = untangler.get_batch_result("batch_67ba558986f08190bb2c61a32d87a2e3")
print("Accuracy: ", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))

100%|██████████| 266/266 [00:00<?, ?it/s]


Accuracy:  0.7819548872180451
              precision    recall  f1-score   support

       Buggy       0.73      0.94      0.82       144
    NotBuggy       0.89      0.60      0.72       122

    accuracy                           0.78       266
   macro avg       0.81      0.77      0.77       266
weighted avg       0.81      0.78      0.77       266

